In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [60]:
def seperate_date_time(x):
    date = pd.to_datetime(x.split(', ')[0],format ="%m/%d/%y").date()
    time = x.split(', ')[1].split(' - ')[0]
    return date, time

In [65]:
def createDf(file_data, dt):
    patterns = {
        '12hr':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24hr':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    print(user_message, date_time)
    
#     df = pd.DataFrame({'user_message': user_message, 'date_time': date_time})
#     df[['date','time']] = df['date_time'].apply(lambda x: seperate_date_time(x)).to_list()
#     users = []
#     messages = []
#     for message in df['user_message']:
#         entry = re.split('([\w\W]+?):\s', message)
#         if entry[1:]:
#             users.append(entry[1])
#             messages.append(entry[2])
#         else:
#             users.append('group_notification')
#             messages.append(entry[0])

#     df['user'] = users
#     df['message'] = messages
#     df.drop(columns=['user_message', 'date_time'], inplace=True)
#     return df

In [66]:
f = open('WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')
createDf(f, '24hr')

[] []


In [63]:
f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
createDf(f, '24hr')

,date,time,user,message
0,2022-05-09,15:01,group_notification,Messages and calls are end-to-end encrypted. N...
1,2022-05-26,19:21,Clg Divyansh,Robotics ke notes
2,2022-05-26,19:44,Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...
3,2022-05-26,19:55,Clg Abbas Amjhera,<Media omitted>
4,2022-05-26,20:05,+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻
...,...,...,...,...
1437,2022-08-17,14:13,Clg Amit Pandey,ni
1438,2022-08-17,14:14,+91 87705 37687,Nhi
1439,2022-08-17,14:29,Clg Ashvin birla,Hmara to parso paper h
1440,2022-08-17,14:29,Clg Ashvin birla,Subh 9 bje


In [71]:
x = 0
y = 1
print((x and y) == 0)

True
